# Skills with Claude API (AWS Bedrock)

This notebook demonstrates how to use Skills-like functionality with AWS Bedrock Claude,
including proper loading of skill assets, references, and scripts.

In [1]:
from dotenv import load_dotenv
import os

# Import your custom Bedrock client
from claude_client import ClaudeAPI  # Update with your actual module name

# Import the enhanced skill framework
from bedrock_agent_skills.skill_loader import SkillLoader
from bedrock_agent_skills.claude_code_integration import SkillBasedAgent, ClaudeCodeAgent

_ = load_dotenv()

In [2]:
# Initialize your Bedrock client
# Initialize your Bedrock client
client = ClaudeAPI(profile_arn=os.getenv("CLAUDE_45_INFERENCE_PROFILE_ARN"))

2026-01-31 00:23:17,435 - INFO - Found credentials in environment variables.


## Part 1: Content Creation - Practice Question Generator

### Initialize Skill-Based Agent

In [3]:
# Create agent with skill support
skill_agent = SkillBasedAgent(
    claude_client=client,
    skills_directory="./custom_skills",
    auto_execute=True,
    require_confirmation=False,
    persistent=True,
    workspace_dir="./workspace"
)

print("Skill agent initialized!")

2026-01-31 00:23:24,406 - INFO - Code executor initialized with workspace: C:\Users\RKU47F\Desktop\skills\workspace


Skill agent initialized!


In [4]:
# List available skills
skills = skill_agent.list_skills()
print("Available Skills:")
print("=" * 60)

for skill in skills:
    info = skill_agent.get_skill_info(skill)
    print(f"\n📁 {skill}")
    print(f"   References: {info['references']}")
    print(f"   Has scripts: {info['has_scripts']}")
    print(f"   Assets: {len(info['assets'])} file(s)")

Available Skills:

📁 analyzing-time-series
   References: ['interpretation.md']
   Has scripts: True
   Assets: 0 file(s)

📁 generating-practice-questions
   References: ['examples_by_topic.md']
   Has scripts: False
   Assets: 1 file(s)


### Load and Inspect the Practice Question Generator Skill

In [5]:
# Load the skill to see what it contains
skill_data = skill_agent.load_skill(
    skill_name="generating-practice-questions",
    include_scripts=False
)

print("Skill Contents:")
print("=" * 60)
print(f"Instructions: {len(skill_data['instructions']):,} characters")
print(f"\nReference files: {len(skill_data['references'])}")
for ref_name in skill_data['references'].keys():
    print(f"  - {ref_name}")
print(f"\nScripts: {len(skill_data['scripts'])}")
print(f"Assets: {len(skill_data['assets'])}")

2026-01-31 00:23:42,393 - INFO - Loaded reference: examples_by_topic.md
2026-01-31 00:23:42,395 - INFO - Loaded skill 'generating-practice-questions': 1 refs, 0 scripts, 1 assets


Skill Contents:
Instructions: 5,553 characters

Reference files: 1
  - examples_by_topic.md

Scripts: 0
Assets: 1


### Execute with Skill (All Assets Loaded)

In [6]:
# Execute with full skill context
result = skill_agent.execute_with_skill(
    skill_name="generating-practice-questions",
    prompt="Each code block must be fully self-contained and include all required imports. Generate practice questions in Markdown format from these lecture notes",
    file_path="./lecture_notes/notes04.tex",
    include_references=True,  # Include all reference files
    include_scripts=False,    # Don't need script contents for this task
    max_tokens=4096
)

print("\n" + "=" * 60)
print("EXECUTION RESULTS")
print("=" * 60)
print(f"Success: {result['success']}")
print(f"Code blocks found: {len(result['code_blocks'])}")
print(f"Files generated: {result['files']}")

2026-01-31 00:23:47,714 - INFO - Using cached skill: generating-practice-questions
2026-01-31 00:23:47,715 - INFO - Sending request to Claude with skill instructions...
2026-01-31 00:24:19,187 - INFO - Received response (9832 chars)
2026-01-31 00:24:19,189 - INFO - Found 3 code blocks
2026-01-31 00:24:19,189 - INFO - Executing code blocks...
2026-01-31 00:24:19,189 - INFO - Executing code block 1/3
2026-01-31 00:24:19,190 - INFO - Found 3 packages to install: ['scikit-learn', 'matplotlib', 'numpy']
2026-01-31 00:24:19,191 - INFO - Installing package: scikit-learn
2026-01-31 00:24:21,503 - INFO - Successfully installed: scikit-learn
2026-01-31 00:24:21,504 - INFO - Installing package: matplotlib
2026-01-31 00:24:23,763 - INFO - Successfully installed: matplotlib
2026-01-31 00:24:23,764 - INFO - Installing package: numpy
2026-01-31 00:24:26,048 - INFO - Successfully installed: numpy
2026-01-31 00:24:29,256 - INFO - Executing code block 2/3
2026-01-31 00:24:29,500 - WARNING - Block 2 fail


EXECUTION RESULTS
Success: True
Code blocks found: 3
Files generated: []


In [7]:
for i, block in enumerate(result['code_blocks'], 1):
    print(f"\n--- block {i} ---\n{block}\n")



--- block 1 ---
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression

def predict(x, w1, w0):
    """
    Calculate predicted values using linear model.
    
    Args:
        x: Feature values (numpy array)
        w1: Slope parameter
        w0: Intercept parameter
    
    Returns:
        Predicted values (numpy array)
    """
    pass

def sum_squared_error(y_true, y_pred):
    """
    Calculate sum of squared errors.
    
    Args:
        y_true: Actual labels (numpy array)
        y_pred: Predicted labels (numpy array)
    
    Returns:
        Sum of squared errors (float)
    """
    pass

def fit_linear_regression(x, y):
    """
    Find optimal slope and intercept using closed-form solution.
    
    Args:
        x: Feature values (numpy array)
        y: Label values (numpy array)
    
    Returns:
        Tuple of (w1, w0) - slope and intercept
    """
    pass


--- block 2 ---
# Generate sample data
np.random.seed(42)
x 

In [8]:
# Display execution details
if result['executions']:
    print("\nExecution Details:")
    print("=" * 60)
    for i, exec_result in enumerate(result['executions'], 1):
        print(f"\nBlock {i}:")
        print(f"  Success: {exec_result['success']}")
        print(f"  Execution time: {exec_result['execution_time']:.2f}s")
        if exec_result['output']:
            print(f"  Output: {exec_result['output'][:200]}...")
        if exec_result['error']:
            print(f"  Error: {exec_result['error']}")


Execution Details:

Block 1:
  Success: True
  Execution time: 10.07s

Block 2:
  Success: False
  Execution time: 0.24s
  Error: Traceback (most recent call last):
  File "C:\Users\RKU47F\Desktop\skills\workspace\execution_script.py", line 2, in <module>
    np.random.seed(42)
    ^^
NameError: name 'np' is not defined


Block 3:
  Success: True
  Execution time: 5.48s
  Output:           GRE       TOEFL  Admitted
0  317.450712  109.261775         1
1  307.926035  119.094166         1
2  319.715328   86.014324         0
3  332.845448  105.629692         0
4  306.487699   93.4...


In [10]:
# Display Claude's response
from IPython.display import Markdown, display

if result['response']:
    print("\nClaude's Response:")
    print("=" * 60)
    display(Markdown(result['response']))


Claude's Response:


# Practice Questions: Machine Learning Models

## Instructions

This practice question set is based on the lecture notes on Machine Learning Models. The questions are designed to test your understanding of the key concepts covered in the lecture.

**Question Types:**
- **Part 1**: True/False Questions - Test your understanding of fundamental concepts
- **Part 2**: Explanatory Questions - Demonstrate deeper understanding through written explanations
- **Part 3**: Coding Question - Implement concepts in Python
- **Part 4**: Use Case Application - Apply concepts to a realistic scenario

**Guidelines:**
- Answer all questions to the best of your ability
- For explanatory questions, provide detailed answers (3-5 sentences)
- For coding questions, ensure your code is well-commented and follows the instructions
- For the use case, show your complete solution with explanations

---

## Part 1: True/False Questions

1. A model is a mathematical tool that provides a perfect representation of real-world phenomena.

2. In supervised machine learning, a model is a mathematical mapping that transforms a feature vector into its corresponding predicted label.

3. Parametric machine learning models assume a specific functional form that depends on a finite number of parameters, regardless of the training dataset size.

4. Non-parametric models have no parameters at all.

5. In a linear regression model with two features, the equation $y = w_1x_1 + w_2x_2 + w_0$ represents a plane in three-dimensional space.

6. Linear classification models use the equation $\textbf{x}^T\textbf{w} + w_0 = 0$ to define a hyperplane that separates the feature space into regions corresponding to different classes.

7. The weight vector $\textbf{w}$ in a linear classification model represents the normal vector that defines the orientation of the separating hyperplane.

8. Training a linear model means finding the optimal weights that minimize the overall error on the training dataset.

---

## Part 2: Explanatory Questions

1. **Explain the relationship between model input, model parameters, and model output in the context of machine learning models.** How do these three components interact, and what role does each play in making predictions?

2. **Compare and contrast parametric and non-parametric machine learning models.** What are the key differences in their assumptions, and how does the amount of training data affect each type?

3. **Describe the geometric interpretation of linear regression models.** How does the dimensionality of the feature space affect this geometric representation, and what does the fitted model represent?

4. **Explain why linear models might not be suitable for all machine learning problems.** Provide examples of scenarios where linear models would struggle, and discuss what characteristics of data make them appropriate or inappropriate for linear modeling.

5. **Describe how the training process for linear models works.** What is being optimized, and how does this optimization lead to finding the best model parameters?

---

## Part 3: Coding Question

**Question: Implement Simple Linear Regression from Scratch**

Implement a simple linear regression model that finds the optimal weights $w_1$ (slope) and $w_0$ (intercept) by minimizing the sum of squared errors.

**Instructions:**

1. Implement a function that calculates the predicted values given features, slope, and intercept
2. Implement a function that computes the sum of squared errors (SSE) between predictions and actual labels
3. Implement a function that finds the optimal slope and intercept using the closed-form solution:
   - $w_1 = \frac{\sum_{i=1}^{n}(x_i - \bar{x})(y_i - \bar{y})}{\sum_{i=1}^{n}(x_i - \bar{x})^2}$
   - $w_0 = \bar{y} - w_1\bar{x}$
4. Test your implementation on sample data and visualize the fitted line
5. Compare your results with sklearn's LinearRegression

**Function Signatures:**

```python
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression

def predict(x, w1, w0):
    """
    Calculate predicted values using linear model.
    
    Args:
        x: Feature values (numpy array)
        w1: Slope parameter
        w0: Intercept parameter
    
    Returns:
        Predicted values (numpy array)
    """
    pass

def sum_squared_error(y_true, y_pred):
    """
    Calculate sum of squared errors.
    
    Args:
        y_true: Actual labels (numpy array)
        y_pred: Predicted labels (numpy array)
    
    Returns:
        Sum of squared errors (float)
    """
    pass

def fit_linear_regression(x, y):
    """
    Find optimal slope and intercept using closed-form solution.
    
    Args:
        x: Feature values (numpy array)
        y: Label values (numpy array)
    
    Returns:
        Tuple of (w1, w0) - slope and intercept
    """
    pass
```

**Test Data:**

```python
# Generate sample data
np.random.seed(42)
x = np.array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
y = 2.5 * x + 3 + np.random.randn(10) * 2  # y = 2.5x + 3 + noise
```

**Expected Output:**

Your implementation should:
- Calculate slope (w1) approximately equal to 2.5
- Calculate intercept (w0) approximately equal to 3
- Produce a plot showing the data points and fitted line
- Match sklearn's LinearRegression results closely

**Hints:**

- Use `np.mean()` to calculate the mean of x and y
- The closed-form solution minimizes the sum of squared errors analytically
- Visualize your results to verify the line fits the data well
- The sum of squared errors should be minimized at the optimal parameters

---

## Part 4: Use Case Application

**Question: Predicting Student Graduate School Admission**

**Scenario:**

You are working as a data scientist for a university's admissions office. The office wants to develop a predictive model to help identify which applicants are likely to be admitted to graduate school based on their test scores. This model will help the admissions committee prioritize application reviews and provide applicants with preliminary feedback.

The admissions office has historical data on previous applicants, including their GRE scores (Graduate Record Examination), TOEFL scores (Test of English as a Foreign Language), and whether they were ultimately admitted.

**Data Generation:**

```python
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Set random seed for reproducibility
np.random.seed(42)

# Generate synthetic admission data
n_samples = 500

# GRE scores (260-340 range)
gre_scores = np.random.normal(310, 15, n_samples)
gre_scores = np.clip(gre_scores, 260, 340)

# TOEFL scores (80-120 range)
toefl_scores = np.random.normal(100, 10, n_samples)
toefl_scores = np.clip(toefl_scores, 80, 120)

# Create admission probability based on scores
# Higher GRE and TOEFL scores increase admission probability
admission_prob = 1 / (1 + np.exp(-(0.05 * gre_scores + 0.08 * toefl_scores - 25)))

# Generate admission decisions
admitted = (np.random.random(n_samples) < admission_prob).astype(int)

# Create DataFrame
df = pd.DataFrame({
    'GRE': gre_scores,
    'TOEFL': toefl_scores,
    'Admitted': admitted
})

print(df.head())
print(f"\nAdmission rate: {df['Admitted'].mean():.2%}")
```

**Task:**

1. **Data Exploration**: Create a scatter plot showing GRE vs TOEFL scores, with different colors for admitted and not admitted students. What patterns do you observe?

2. **Data Preprocessing**: Split the data into training (70%) and testing (30%) sets. Should you standardize the features? Explain your reasoning.

3. **Model Training**: Train a logistic regression model (a linear classification model) to predict admission based on GRE and TOEFL scores.

4. **Visualization**: Plot the decision boundary created by your linear model in the feature space. Explain what this boundary represents geometrically.

5. **Model Evaluation**: 
   - Calculate and report the accuracy on both training and test sets
   - Create a confusion matrix for the test set
   - Discuss whether the linear model is appropriate for this problem

6. **Interpretation**: 
   - Extract and interpret the model coefficients (weights)
   - Which test score (GRE or TOEFL) has a stronger influence on admission decisions according to your model?
   - For a student with GRE = 320 and TOEFL = 105, what is the predicted probability of admission?

7. **Limitations**: Discuss the limitations of using a linear model for this admission prediction task. What real-world factors are not captured by this simple model?

**Requirements:**

- Use scikit-learn's `LogisticRegression` for the linear classification model
- Include all necessary visualizations with proper labels and legends
- Provide clear explanations for each step of your analysis
- Comment your code thoroughly

**Hints:**

- For plotting the decision boundary, create a mesh grid of points in the feature space and predict the class for each point
- The decision boundary is where the model's prediction probability equals 0.5
- Remember that logistic regression coefficients represent the change in log-odds for a one-unit change in the feature
- Consider whether the relationship between test scores and admission is truly linear
- Think about what the normal vector $\textbf{w}$ represents in the context of this problem

**Deliverables:**

1. Complete Python code with all implementations
2. Visualizations (scatter plot with classes, decision boundary plot)
3. Model performance metrics and interpretation
4. Written analysis addressing all questions in the task section

---

**End of Practice Questions**

### View Generated Files

In [11]:
# List all generated files
all_files = skill_agent.list_files()
print(f"Generated files ({len(all_files)}):")
for f in all_files:
    print(f"  - {f}")

Generated files (0):


In [12]:
# View the generated markdown file
if result['files']:
    for filename in result['files']:
        if filename.endswith('.md'):
            content = skill_agent.get_file(filename)
            if content:
                print(f"\nContent of {filename}:")
                print("=" * 60)
                display(Markdown(content.decode('utf-8')))

## Part 2: Data Analysis - Time Series Analysis

### Inspect Time Series Analysis Skill

In [20]:
# Check what assets the time series skill has
ts_info = skill_agent.get_skill_info("analyzing-time-series")

print("Time Series Analysis Skill:")
print("=" * 60)
print(f"Path: {ts_info['path']}")
print(f"Reference files: {ts_info['references']}")
print(f"Has scripts: {ts_info['has_scripts']}")
print(f"Assets: {ts_info['assets']}")

if ts_info.get('metadata'):
    print(f"\nMetadata:")
    for key, value in ts_info['metadata'].items():
        print(f"  {key}: {value}")

Time Series Analysis Skill:
Path: custom_skills\analyzing-time-series
Reference files: ['interpretation.md']
Has scripts: True
Assets: []

Metadata:
  name: analyzing-time-series
  description: Comprehensive diagnostic analysis of time series data. Use when users provide CSV time series data and want to understand its characteristics before forecasting - stationarity, seasonality, trend, forecastability, and transform recommendations.


### Execute Time Series Analysis

In [21]:
query = """
Analyze this time series data and create:
1. Diagnostic plots (trend, seasonality, residuals)
2. Statistical analysis
3. A Word document report summarizing findings

Use the reference materials and best practices from the skill.
"""

In [22]:
# Execute with full skill context including references
result = skill_agent.execute_with_skill(
    skill_name="analyzing-time-series",
    prompt=query,
    file_path="./data/retail_sales.csv",
    include_references=True,  # Include reference documentation
    include_scripts=True,     # Include helper scripts for this complex task
    max_tokens=16384
)

print("\n" + "=" * 60)
print("TIME SERIES ANALYSIS RESULTS")
print("=" * 60)
print(f"Success: {result['success']}")
print(f"Code blocks: {len(result['code_blocks'])}")
print(f"Files generated: {result['files']}")

2026-01-31 00:48:52,874 - INFO - Using cached skill: analyzing-time-series
2026-01-31 00:48:52,875 - INFO - Sending request to Claude with skill instructions...
2026-01-31 00:50:42,679 - INFO - Received response (35219 chars)
2026-01-31 00:50:42,681 - INFO - Found 5 code blocks
2026-01-31 00:50:42,681 - INFO - Executing code blocks...
2026-01-31 00:50:42,682 - INFO - Executing code block 1/5
2026-01-31 00:50:42,683 - WARNING - Block 1 failed: Safety check failed:
Blocked import detected: 'subprocess'
Dangerous pattern detected: subprocess\.
2026-01-31 00:50:42,683 - INFO - Executing code block 2/5
2026-01-31 00:50:42,683 - WARNING - Block 2 failed: Safety check failed:
Dangerous pattern detected: subprocess\.
2026-01-31 00:50:42,684 - INFO - Executing code block 3/5
2026-01-31 00:50:42,686 - INFO - Found 2 packages to install: ['pathlib', 'docx']
2026-01-31 00:50:42,687 - INFO - Package 'pathlib' already installed
2026-01-31 00:50:42,687 - INFO - Package 'docx' already installed
2026-0


TIME SERIES ANALYSIS RESULTS
Success: True
Code blocks: 5
Files generated: []


In [23]:
print(result['response'])

I'll analyze this time series data comprehensively using the diagnostic tools. Let me start by saving the data and running the analysis.

<write_file>
<path>sales_data.csv</path>
<content>date,sales
2015-01-01,46743
2015-02-01,68008
2015-03-01,71943
2015-04-01,62245
2015-05-01,56679
2015-06-01,58467
2015-07-01,46249
2015-08-01,52259
2015-09-01,52441
2015-10-01,37693
2015-11-01,33927
2015-12-01,66758
2016-01-01,61532
2016-02-01,70158
2016-03-01,76821
2016-04-01,72520
2016-05-01,72091
2016-06-01,67132
2016-07-01,63600
2016-08-01,61763
2016-09-01,57914
2016-10-01,51825
2016-11-01,40215
2016-12-01,77628
2017-01-01,60356
2017-02-01,77220
2017-03-01,87934
2017-04-01,76596
2017-05-01,72112
2017-06-01,65045
2017-07-01,66880
2017-08-01,59267
2017-09-01,57446
2017-10-01,52312
2017-11-01,52863
2017-12-01,80639
2018-01-01,71185
2018-02-01,88257
2018-03-01,89632
2018-04-01,88792
2018-05-01,77175
2018-06-01,69506
2018-07-01,73533
2018-08-01,76443
2018-09-01,70835
2018-10-01,61435
2018-11-01,64317
20

In [24]:
# Show execution summary
if result['executions']:
    print("\nExecution Summary:")
    print("=" * 60)
    
    successful = sum(1 for e in result['executions'] if e['success'])
    print(f"Total executions: {len(result['executions'])}")
    print(f"Successful: {successful}")
    print(f"Failed: {len(result['executions']) - successful}")
    
    for i, exec_result in enumerate(result['executions'], 1):
        print(f"\n--- Block {i} ---")
        print(f"Success: {exec_result['success']}")
        print(f"Time: {exec_result['execution_time']:.2f}s")
        
        if exec_result['output']:
            print(f"Output:\n{exec_result['output']}")
        
        if exec_result['error']:
            print(f"Error: {exec_result['error']}")


Execution Summary:
Total executions: 5
Successful: 0
Failed: 5

--- Block 1 ---
Success: False
Time: 0.00s
Error: Safety check failed:
Blocked import detected: 'subprocess'
Dangerous pattern detected: subprocess\.

--- Block 2 ---
Success: False
Time: 0.00s
Error: Safety check failed:
Dangerous pattern detected: subprocess\.

--- Block 3 ---
Success: False
Time: 0.01s
Error: Execution error: 'charmap' codec can't encode character '\u2713' in position 1043: character maps to <undefined>
Traceback (most recent call last):
  File "c:\Users\RKU47F\Desktop\skills\bedrock_agent_skills\code_executor.py", line 301, in execute_python_code
    f.write(code)
    ~~~~~~~^^^^^^
  File "C:\Python313\Lib\encodings\cp1252.py", line 19, in encode
    return codecs.charmap_encode(input,self.errors,encoding_table)[0]
           ~~~~~~~~~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
UnicodeEncodeError: 'charmap' codec can't encode character '\u2713' in position 1043: character maps to <undefined>


--- 

### Display Generated Visualizations

In [25]:
# Display any generated images
from IPython.display import Image
import os

workspace = skill_agent.executor.workspace_dir

print("Generated Visualizations:")
print("=" * 60)

for filename in result['files']:
    if filename.endswith(('.png', '.jpg', '.jpeg')):
        img_path = os.path.join(workspace, filename)
        if os.path.exists(img_path):
            print(f"\n{filename}:")
            display(Image(filename=img_path))

Generated Visualizations:


### View Generated Report

In [26]:
# Display the Word document report (if it was saved as markdown)
for filename in result['files']:
    if filename.endswith('.docx'):
        print(f"\nWord document generated: {filename}")
        print(f"Location: {os.path.join(workspace, filename)}")
        print("\nDownload this file to view the full report.")
    elif filename.endswith('.md') and 'report' in filename.lower():
        content = skill_agent.get_file(filename)
        if content:
            print(f"\nReport ({filename}):")
            print("=" * 60)
            display(Markdown(content.decode('utf-8')))

## Part 3: Understanding Skill Asset Loading

### Compare Different Loading Strategies

In [18]:
# Initialize a skill loader to inspect prompts
loader = SkillLoader("./custom_skills")

if skills:
    skill_name = skills[0]
    
    # Build prompts with different levels of asset inclusion
    prompt_minimal = loader.build_comprehensive_prompt(
        skill_name=skill_name,
        user_query="Test query",
        include_references=False,
        include_scripts=False
    )
    
    prompt_with_refs = loader.build_comprehensive_prompt(
        skill_name=skill_name,
        user_query="Test query",
        include_references=True,
        include_scripts=False
    )
    
    prompt_full = loader.build_comprehensive_prompt(
        skill_name=skill_name,
        user_query="Test query",
        include_references=True,
        include_scripts=True
    )
    
    print(f"Skill: {skill_name}")
    print("=" * 60)
    print(f"\nPrompt Size Comparison:")
    print(f"  Minimal (SKILL.md only):     {len(prompt_minimal):>8,} chars (~{len(prompt_minimal)//4:>6,} tokens)")
    print(f"  With references:             {len(prompt_with_refs):>8,} chars (~{len(prompt_with_refs)//4:>6,} tokens) [+{len(prompt_with_refs)-len(prompt_minimal):,}]")
    print(f"  Full (refs + scripts):       {len(prompt_full):>8,} chars (~{len(prompt_full)//4:>6,} tokens) [+{len(prompt_full)-len(prompt_minimal):,}]")
    
    print(f"\nWhat's included in full prompt:")
    sections = [
        ('<skill_instructions>', 'Main SKILL.md'),
        ('<skill_reference_files>', 'Reference .md files'),
        ('<skill_scripts>', 'Python scripts'),
        ('<skill_assets>', 'Asset listing'),
    ]
    
    for tag, description in sections:
        if tag in prompt_full:
            print(f"  ✓ {description}")
        else:
            print(f"  ✗ {description}")

2026-01-30 23:47:12,670 - INFO - Loaded reference: interpretation.md
2026-01-30 23:47:12,672 - INFO - Loaded skill 'analyzing-time-series': 1 refs, 0 scripts, 0 assets
2026-01-30 23:47:12,673 - INFO - Using cached skill: analyzing-time-series
2026-01-30 23:47:12,673 - INFO - Using cached skill: analyzing-time-series


Skill: analyzing-time-series

Prompt Size Comparison:
  Minimal (SKILL.md only):        3,171 chars (~   792 tokens)
  With references:                9,200 chars (~ 2,300 tokens) [+6,029]
  Full (refs + scripts):          9,200 chars (~ 2,300 tokens) [+6,029]

What's included in full prompt:
  ✓ Main SKILL.md
  ✓ Reference .md files
  ✗ Python scripts
  ✗ Asset listing


### Best Practices for Asset Loading

In [19]:
print("When to Include Skill Assets:")
print("=" * 60)

print("""
✅ INCLUDE REFERENCES (include_references=True) when:
   - Skill has additional documentation (editing.md, pptxgenjs.md, etc.)
   - Task requires advanced features described in references
   - You need Claude to follow specific patterns/examples
   - Token budget allows (adds ~20-50K tokens)

✅ INCLUDE SCRIPTS (include_scripts=True) when:
   - Task might benefit from existing utilities
   - You want Claude to understand/use helper functions
   - Complex task needs comprehensive context
   - Token budget allows (adds ~10-30K tokens)

❌ SKIP THEM when:
   - Simple, straightforward task
   - SKILL.md alone provides sufficient guidance
   - Token budget is constrained
   - Rapid iteration/exploration phase

💡 EXAMPLES:

Simple task:
   skill_agent.execute_with_skill(
       skill_name="docx",
       prompt="Create a simple document with title and text",
       include_references=False,  # Basic SKILL.md is enough
       include_scripts=False
   )

Complex task:
   skill_agent.execute_with_skill(
       skill_name="docx",
       prompt="Create document with tracked changes, comments, and custom formatting",
       include_references=True,   # Need editing.md for advanced features
       include_scripts=True        # Need accept_changes.py and comment.py
   )
""")

When to Include Skill Assets:

✅ INCLUDE REFERENCES (include_references=True) when:
   - Skill has additional documentation (editing.md, pptxgenjs.md, etc.)
   - Task requires advanced features described in references
   - You need Claude to follow specific patterns/examples
   - Token budget allows (adds ~20-50K tokens)

✅ INCLUDE SCRIPTS (include_scripts=True) when:
   - Task might benefit from existing utilities
   - You want Claude to understand/use helper functions
   - Complex task needs comprehensive context
   - Token budget allows (adds ~10-30K tokens)

❌ SKIP THEM when:
   - Simple, straightforward task
   - SKILL.md alone provides sufficient guidance
   - Token budget is constrained
   - Rapid iteration/exploration phase

💡 EXAMPLES:

Simple task:
   skill_agent.execute_with_skill(
       skill_name="docx",
       prompt="Create a simple document with title and text",
       include_references=False,  # Basic SKILL.md is enough
       include_scripts=False
   )

Complex task

## Part 4: Advanced Workflows

### Multi-Skill Workflow

In [20]:
# Example: Combine multiple skills in a workflow
workflow_agent = ClaudeCodeAgent(
    claude_client=client,
    persistent=True,
    workspace_dir="./workflow_workspace",
    auto_execute=True,
    require_confirmation=False
)

# Define multi-step workflow
workflow_tasks = [
    {
        'prompt': """
        Step 1: Analyze the time series data
        Load ./data/retail_sales.csv and perform analysis.
        Save key findings to findings.txt
        """,
        'max_tokens': 6000
    },
    {
        'prompt': """
        Step 2: Create visualizations
        Based on the previous analysis, create:
        - Trend plot
        - Seasonality plot
        - Forecast plot
        Save as PNG files.
        """,
        'max_tokens': 4000
    },
    {
        'prompt': """
        Step 3: Generate comprehensive report
        Create a Word document that includes:
        - Executive summary
        - Analysis findings from findings.txt
        - References to the generated plots
        - Recommendations
        """,
        'max_tokens': 5000
    }
]

print("Executing multi-step workflow...")
print("=" * 60)

workflow_results = workflow_agent.execute_workflow(
    tasks=workflow_tasks,
    share_context=True  # Each step gets context from previous steps
)

print("\nWorkflow Complete!")
print("=" * 60)
print(f"Total steps: {len(workflow_results)}")

for i, result in enumerate(workflow_results, 1):
    print(f"\nStep {i}:")
    print(f"  Success: {result['success']}")
    print(f"  Files: {result['files']}")
    if result['executions']:
        successful = sum(1 for e in result['executions'] if e['success'])
        print(f"  Executions: {successful}/{len(result['executions'])} successful")

2026-01-30 23:47:12,696 - INFO - Code executor initialized with workspace: C:\Users\RKU47F\Desktop\skills\workflow_workspace
2026-01-30 23:47:12,697 - INFO - 
2026-01-30 23:47:12,698 - INFO - Task 1/3: 
        Step 1: Analyze the time series data
    ...
2026-01-30 23:47:12,698 - INFO - ============================================================
2026-01-30 23:47:12,699 - INFO - Sending request to Claude...


Executing multi-step workflow...


2026-01-30 23:47:36,069 - INFO - Received response (8169 chars)
2026-01-30 23:47:36,070 - INFO - Found 11 code blocks
2026-01-30 23:47:36,071 - INFO - Executing code blocks...
2026-01-30 23:47:36,072 - INFO - Executing code block 1/11
2026-01-30 23:47:36,072 - INFO - Found 4 packages to install: ['matplotlib', 'numpy', 'pandas', 'seaborn']
2026-01-30 23:47:36,073 - INFO - Installing package: matplotlib
2026-01-30 23:47:38,412 - INFO - Successfully installed: matplotlib
2026-01-30 23:47:38,413 - INFO - Installing package: numpy
2026-01-30 23:47:40,768 - INFO - Successfully installed: numpy
2026-01-30 23:47:40,769 - INFO - Installing package: pandas
2026-01-30 23:47:43,060 - INFO - Successfully installed: pandas
2026-01-30 23:47:43,061 - INFO - Installing package: seaborn
2026-01-30 23:47:46,087 - INFO - Successfully installed: seaborn
2026-01-30 23:47:48,997 - WARNING - Block 1 failed: Traceback (most recent call last):
  File "C:\Users\RKU47F\Desktop\skills\workflow_workspace\execution


Workflow Complete!
Total steps: 3

Step 1:
  Success: True
  Files: ['findings.txt']
  Executions: 0/11 successful

Step 2:
  Success: True
  Files: []
  Executions: 0/4 successful

Step 3:
  Success: True
  Files: []


In [21]:
# View all files generated by the workflow
all_workflow_files = workflow_agent.list_files()

print(f"\nAll Generated Files ({len(all_workflow_files)}):")
print("=" * 60)
for f in all_workflow_files:
    print(f"  - {f}")


All Generated Files (1):
  - findings.txt


## Part 5: Skill Development Tips

### Recommended Skill Directory Structure

```
custom_skills/
├── your-skill-name/
│   ├── SKILL.md              # Main instructions (REQUIRED)
│   ├── reference1.md         # Additional documentation
│   ├── reference2.md         # More examples/patterns
│   ├── scripts/              # Helper scripts
│   │   ├── __init__.py
│   │   ├── helper1.py
│   │   └── helper2.py
│   └── assets/               # Templates, examples, etc.
│       ├── template.docx
│       └── example.json
```

### SKILL.md Format

```markdown
---
name: your-skill-name
description: Brief description of what this skill does
license: MIT
---

# Skill Name

## Overview
What this skill does...

## Quick Reference
Common patterns and usage...

## Examples
Code examples...

## Best Practices
Tips and recommendations...
```

## Cleanup (Optional)

In [22]:
# Uncomment to clean up workspaces
# skill_agent.cleanup()
# workflow_agent.cleanup()

print("Note: Workspaces are persistent and not cleaned up by default.")
print(f"Skill agent workspace: {skill_agent.executor.workspace_dir}")
print(f"Workflow agent workspace: {workflow_agent.executor.workspace_dir}")

Note: Workspaces are persistent and not cleaned up by default.
Skill agent workspace: C:\Users\RKU47F\Desktop\skills\workspace
Workflow agent workspace: C:\Users\RKU47F\Desktop\skills\workflow_workspace


## Summary

### Key Differences from Anthropic Skills API:

| Feature | Anthropic API | This Implementation |
|---------|--------------|---------------------|
| Skill Upload | API-based | Local directory |
| File Upload | API-based | Local file loading |
| Code Execution | Managed sandbox | Local subprocess |
| File Download | API-based | Direct file access |
| Asset Loading | Automatic | Manual via SkillLoader |
| References | Automatic | Controlled via flags |

### Advantages of This Approach:

✅ **Full Control**: You manage execution environment and security
✅ **AWS Integration**: Native AWS Bedrock integration
✅ **Flexibility**: Granular control over asset loading
✅ **Cost**: Potentially more economical for high-volume use
✅ **Customization**: Easy to extend and modify

### Best Practices:

1. **Start Simple**: Use `include_references=False, include_scripts=False` for basic tasks
2. **Scale Up**: Add references and scripts as task complexity increases
3. **Monitor Tokens**: Be mindful of context window usage
4. **Review Code**: Always review generated code before execution in production
5. **Use Persistent Workspaces**: For multi-step workflows
6. **Cache Skills**: The SkillLoader caches loaded skills automatically

### Next Steps:

- Create your own custom skills
- Experiment with different asset loading strategies
- Build complex multi-skill workflows
- Integrate with your existing AWS infrastructure